In [13]:
import basic_image_operations as basic_ops
import filters
import geometrical_image_operations as geo_ops
import traffic_sign_detection as tsd
import numpy as np

bgr_img = basic_ops.load_image(image_path='traffic_sign_images/stop_01.jpg')
bgr_img = geo_ops.resize_image(image=bgr_img, target_height=bgr_img.sh ape[0]//8, target_width=bgr_img.shape[1]//8)
for y in range(bgr_img.shape[0]):
    for x in range(bgr_img.shape[1]):
        hsv = tsd.bgr_to_hsv(bgr_img[y, x])
        h, s, v = hsv[0], hsv[1], hsv[2]
        if tsd.is_strong_red(h, s, v):
            continue
        bgr_img[y, x] = np.array([0, 0, 0])

erosion_dilation_img = bgr_img
erosion_dilation_img = filters.dilation(image=erosion_dilation_img, dim=9)
erosion_dilation_img = filters.erosion(image=erosion_dilation_img, dim=9)

red_mask = tsd.get_red_mask(erosion_dilation_img)
blobs = tsd.label_connected_components(red_mask)
boxed_image = tsd.draw_bounding_boxes(image=erosion_dilation_img, blobs=blobs, min_box_area=1000)
basic_ops.show_image(image=boxed_image)

Image loaded from traffic_sign_images/stop_01.jpg
Image displayed


In [14]:
import basic_image_operations as basic_ops
import filters

templates = basic_ops.load_images(folder_path='traffic_sign_templates/stop_signs', amount=100)
for i, template in enumerate(templates):
    template = filters.gray_scale_filter(image=template)
    template = filters.sobel_filter(image=template, mode='both')
    templates[i] = template


Image loaded from traffic_sign_templates/stop_signs/stop_sample_resized_2.jpg
Image loaded from traffic_sign_templates/stop_signs/stop_sample_resized_4.jpg
Image loaded from traffic_sign_templates/stop_signs/stop_sample.jpg
Image loaded from traffic_sign_templates/stop_signs/stop_sample_resized_8.jpg


In [15]:
import basic_image_operations as basic_ops
import filters
import geometrical_image_operations as geo_ops
import statistical_operations as stat_ops
import traffic_sign_detection as tsd
import numpy as np

bgr_img = basic_ops.load_image(image_path='traffic_sign_templates/stop_signs/stop_sample.jpg')
basic_ops.show_image(image=bgr_img)
#bgr_img = geo_ops.resize_image(image=bgr_img, target_height=bgr_img.shape[0]//4, target_width=bgr_img.shape[1]//4)
bgr_img = filters.gray_scale_filter(image=bgr_img)
bgr_img = filters.sobel_filter(image=bgr_img, mode='both')
basic_ops.show_image(image=bgr_img)

Image loaded from traffic_sign_templates/stop_signs/stop_sample.jpg
Image displayed
Image displayed


In [16]:
threshold = 0.85
output = np.zeros_like(bgr_img, dtype=np.uint8)
for template in templates:
    template_height, template_width = template.shape
    template_white, template_black = np.sum(template == 255), np.sum(template == 0)
    basic_ops.show_image(image=template)
    for y in range(bgr_img.shape[0]):
        for x in range(bgr_img.shape[1]):
            window = bgr_img[y:y+template_height, x:x+template_width]

            window_white = np.sum(window == 255)
            window_black = np.sum(window == 0)
            similarity = 1 - (abs(template_white - window_white) + abs(template_black - window_black)) / (template_white + template_black)

            if similarity > threshold:
                output[y+template_height//2, x+template_width//2] += 255/len(templates)

    basic_ops.show_image(image=output)
    basic_ops.show_image(image=bgr_img)

max_val = 0
max_indices = None, None
for y in range(output.shape[0]):
    for x in range(output.shape[1]):
        if output[y, x] > max_val:
            max_val = output[y, x]
            max_indices = y, x
print(f'max value is {max_val} at {max_indices}')






Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
max value is 126 at (161, 185)
